- 네이버 영화 평점: https://movie.naver.com/movie/point/af/list.nhn
- 훈련 데이터: ratings_train.txt
- Test 데이터: ratings_test.txt
- 데이터: https://github.com/e9t/nsmc
    - 리뷰: document
    - 긍부정: label - 0:부정, 1:긍정

## 훈련/테스트 데이터 셋 읽기

In [6]:
import pandas as pd  # 파일 확인후 구분좌가 뭔지 확인
train_df = pd.read_csv('data/ratings_train.txt',sep = '\t', encoding = 'utf-8')
test_df = pd.read_csv('data/ratings_test.txt',sep = '\t', encoding = 'utf-8')

train_df.shape, test_df.shape

((150000, 3), (50000, 3))

## 데이터셋 확인

In [7]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [8]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [11]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [18]:
train_df.isna().sum()  # 5개 제거함 

id          0
document    0
label       0
dtype: int64

In [19]:
test_df.isna().sum()# 3 개 제거함

id          0
document    0
label       0
dtype: int64

In [15]:
train_df[train_df['document'].isna()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [16]:
test_df[test_df['document'].isna()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [ ]:
# 결측치 제거

In [17]:
train_df.dropna(inplace = True)
test_df.dropna(inplace= True)

In [20]:
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [21]:
test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

## 텍스트 전처리

In [25]:
string.punctuation  # 특수문자 반환/ (구두점..)

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [27]:
import string 
import re
pattern = f'[{string.punctuation }]'
re.sub(pattern, " ", '가나다라!%^*&^%$#,마바!사') # pattern 의 문자들이 나오면 " "으로 바꾼다.


'가나다라          마바 사'

In [28]:
import string, re
from konlpy.tag import Okt
# 특정 품사 (형용사, 부사, 동사, 명사, 감탄사)의 token 추출
# N글자 이상의 token만 추출..  시도해볼 수 있다.

def text_preprocessing(document):
    """document: 댓글 하나."""
    okt = Okt()
    
    # 특수문자 제거
    pattern = f'[{string.punctuation }]'
    document = re.sub(pattern, ' ', document)
    
    
    # 형태소 토큰화
    
    tokens = okt.morphs(document, norm = True)
    
    # 불용어제거... 
    # pass 함.. 
    
    return ' '.join(tokens)

In [32]:
train_df.loc[3,'document']

'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [31]:
text_preprocessing(train_df.loc[3,'document'])

'교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정'

In [33]:
okt =Okt()
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [36]:
X_train = train_df['document'].apply(text_preprocessing)  # apply뭐였지...?
X_test = test_df['document'].apply(text_preprocessing)

In [37]:
y_train = train_df['label']
y_test = test_df['label']

In [38]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((149995,), (149995,), (49997,), (49997,))

## Feature Vectorization

In [40]:
#data = X_train+X_test  # 시리즈라서 그냥 더하면  정상적으로는(작동은 되지만 정상적으로는 안됨) 안됨 // 

data = pd.concat([X_train,X_test])
data.shape

(199992,)

In [41]:
data.head()

0                                    아 더빙 진짜 짜증나네요 목소리
1                    흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2                            너 무재 밓었 다그 래서 보는것을 추천 한 다
3                         교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4    사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만 했...
Name: document, dtype: object

In [42]:
data.tail()  # 인덱스는 사용안하니까 그냥 concat을 하면 됨 // 시리즈라서 그냥 concat 하면 되니다..

49995    오랜 만 에 평점 로 긴 했네 ㅋㅋ 킹왕짱 쌈뽕 한 영화 를 만났습니다 강렬하게 육 쾌함
49996     의지 박약 들 이나 하는거다 탈영 은 일단 주인공 김대희 닮았고 이등병 찐 따 OOOO
49997                    그림 도 좋고 완성 도도 높았지만 보는 내내 불안하게 만든다
49998    절대 봐 서는 안 될 영화 재미 도 없고 기분 만 잡 치고 한 세트 장 에서 다 해먹 네
49999                                         마무리 는 또 왜 이래
Name: document, dtype: object

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [66]:
cv = CountVectorizer(min_df = 10,ngram_range=(1,3)) # e데이터 수가 많아서 제한

cv.fit(data)

CountVectorizer(min_df=10, ngram_range=(1, 3))

In [67]:
train_cv = cv.transform(X_train)
test_cv = cv.transform(X_test)

In [68]:
train_cv.shape,test_cv.shape

((149995, 27932), (49997, 27932))

In [69]:
tfidf = TfidfVectorizer(min_df = 10,ngram_range=(1,3))
tfidf.fit(data)

train_tfidf = tfidf.transform(X_train)
test_tfidf = tfidf.transform(X_test)


In [70]:
train_tfidf.shape, test_tfidf.shape

((149995, 27932), (49997, 27932))

In [49]:
train_cv[:3].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [50]:
train_tfidf[:3].toarray()  # 페널티를 부여하니까.. 실수... 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 긍부정 예측 머신러닝 모델링

In [71]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(max_iter = 1000, random_state = 1)

lg.fit(train_cv,y_train)   

LogisticRegression(max_iter=1000, random_state=1)

In [72]:
pred_train = lg.predict(train_cv)
pred_test = lg.predict(test_cv)

In [62]:
from sklearn.metrics import accuracy_score,classification_report

In [73]:
accuracy_score(y_train,pred_train), accuracy_score(y_test,pred_test)

(0.8924297476582552, 0.838150289017341)

In [ ]:

# tfidf 로 예측 

In [74]:
lg2 = LogisticRegression(max_iter = 1000, random_state= 1)
lg2.fit(train_tfidf, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [75]:
pred_train2 = lg2.predict(train_tfidf)
pred_test2 = lg2.predict(test_tfidf)

In [76]:
accuracy_score(y_train, pred_train2), accuracy_score(y_test,pred_test2)

(0.8751625054168473, 0.8422905374322459)

In [ ]:
# 가장 성능이 좋았던 전처리된 data를 만들고.. 머신러닝 하이퍼 파라미터 튜닝을 하는~~!
# 일반적으로 ngram 을 높이면 성능이 좋아지긴 하지만 다른 것들도 시도를 해봐야한다~!


- 그냥 cv cv = CountVectorizer(min_df = 10) 인상태에서 lg1으로 예측 
accuracy_score(y_train,pred_train), accuracy_score(y_test,pred_test)
(0.8715623854128471, 0.8329099745984759)


-  cv = CountVectorizer(min_df = 10,ngram_range=(1,3)) 일때                                      
(0.8924297476582552, 0.838150289017341)                                                                 


- tfidf /tfidf = TfidfVectorizer(min_df = 10) 인상태에서 lg2로 예측 
accuracy_score(y_train, pred_train2), accuracy_score(y_test,pred_test2)
(0.8639087969598986, 0.8356501390083405)


- tfidf = TfidfVectorizer(min_df = 10,ngram_range=(1,3))
accuracy_score(y_train, pred_train2), accuracy_score(y_test,pred_test2)
(0.8751625054168473, 0.8422905374322459)

### 시간은 조금씩 더 오래 걸려지지만 정확성들을 높일 수 있다.~~!